<a href="https://colab.research.google.com/github/sahana0007/Spark/blob/main/DF_Highest%20salary_by%20dept_window_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
from pyspark import SparkContext, SparkConf
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("getNthhighestSalary").master("local[*]").getOrCreate()

#sc = SparkContext("local","PySpark Rank")
#conf = SparkConf().setAppName("Pyspark Project").setMaster("local")
#sc = SparkContext (conf = conf)

Find top 3 salary departmentwise

In [11]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number
from pyspark import SparkContext
from pyspark.sql import SQLContext

#file_rdd = sc.textFile("/content/Salaries1.txt")


In [12]:
df = spark.read\
      .format("csv")\
      .option("header", "true")\
      .option("inferSchema", "true")\
      .csv("/content/Salaries1.csv")
  

In [13]:
df.show()

+---+------------------+--------------------+---------+-----------+---------+--------+---------+----------------+----+-----+-------------+------+------------+
| Id|      EmployeeName|            JobTitle|  BasePay|OvertimePay| OtherPay|Benefits| TotalPay|TotalPayBenefits|Year|Notes|       Agency|Status|DepartmentId|
+---+------------------+--------------------+---------+-----------+---------+--------+---------+----------------+----+-----+-------------+------+------------+
|  1|    NATHANIEL FORD|GENERAL MANAGER-M...|167411.18|        0.0|400184.25|    null|567595.43|       567595.43|2011| null|San Francisco|  null|           7|
|  2|      GARY JIMENEZ|CAPTAIN III (POLI...|155966.02|  245131.88|137811.38|    null|538909.28|       538909.28|2011| null|San Francisco|  null|           7|
|  3|    ALBERT PARDINI|CAPTAIN III (POLI...|212739.13|  106088.18|  16452.6|    null|335279.91|       335279.91|2011| null|San Francisco|  null|           7|
|  4| CHRISTOPHER CHONG|WIRE ROPE CABLE M...| 

In [14]:
from pyspark.sql.functions import col, dense_rank
from pyspark.sql.window import Window




In [15]:
windowfn = Window.partitionBy(col("DepartmentId")).orderBy(col("TotalPay"))

In [16]:
df_salary_rank = df.withColumn("salary_rank", dense_rank().over (windowfn))

In [17]:
df_salary_rank.show()

+---+----------------+--------------------+---------+-----------+---------+--------+---------+----------------+----+-----+-------------+------+------------+-----------+
| Id|    EmployeeName|            JobTitle|  BasePay|OvertimePay| OtherPay|Benefits| TotalPay|TotalPayBenefits|Year|Notes|       Agency|Status|DepartmentId|salary_rank|
+---+----------------+--------------------+---------+-----------+---------+--------+---------+----------------+----+-----+-------------+------+------------+-----------+
| 42|   THOMAS ABBOTT|BATTALION CHIEF, ...|168692.63|    59760.9| 21954.96|    null|250408.49|       250408.49|2011| null|San Francisco|  null|           7|          1|
| 41|MICHAEL THOMPSON|BATTALION CHIEF, ...|168692.66|   60847.56| 21055.17|    null|250595.39|       250595.39|2011| null|San Francisco|  null|           7|          2|
| 40|    BRENDAN WARD|BATTALION CHIEF, ...|147158.85|   64985.85| 39540.46|    null|251685.16|       251685.16|2011| null|San Francisco|  null|           7

In [18]:
n = int(input("Enter the value of n: "))
print("n is: ", n)

df_salary_rank.filter(f"salary_rank == {n}").show()

Enter the value of n: 7
n is:  7
+----+--------------+--------------------+---------+-----------+--------+--------+---------+----------------+----+-----+-------------+------+------------+-----------+
|  Id|  EmployeeName|            JobTitle|  BasePay|OvertimePay|OtherPay|Benefits| TotalPay|TotalPayBenefits|Year|Notes|       Agency|Status|DepartmentId|salary_rank|
+----+--------------+--------------------+---------+-----------+--------+--------+---------+----------------+----+-----+-------------+------+------------+-----------+
|  36|    KHOA TRINH|ELECTRONIC MAINTE...| 101953.2|  139102.95|16483.94|    null|257540.09|       257540.09|2011| null|San Francisco|  null|           7|          7|
| 317|      BOND YEE|DEPUTY DIRECTOR I...| 195256.1|        0.0|   408.0|    null| 195664.1|        195664.1|2011| null|San Francisco|  null|           8|          7|
| 552| DENISE NEWMAN|BATTALION CHIEF, ...| 168713.2|    5340.28|10122.78|    null|184176.26|       184176.26|2011| null|San Francisc

In [19]:
df.createOrReplaceTempView("Employee")

In [33]:
sqlDF = spark.sql("Select EmployeeName, TotalPay, DepartmentId , (rank() over(partition by DepartmentId order by TotalPay DESC)) as rank from Employee")

+----------------+---------+------------+----+
|    EmployeeName| TotalPay|DepartmentId|rank|
+----------------+---------+------------+----+
|       ALSON LEE|315981.05|           7|   7|
|    JAMES DUDLEY|248718.64|           8|   7|
|MICHAEL GONZALES|195232.37|           9|   7|
|   ELISA RAMIREZ|183656.84|          10|   7|
+----------------+---------+------------+----+



In [34]:
n = int(input("Enter the value of n: "))
print("n is: ", n)
sqlDF.filter(f"rank=={n}").show()

Enter the value of n: 5
n is:  5
+----------------+---------+------------+----+
|    EmployeeName| TotalPay|DepartmentId|rank|
+----------------+---------+------------+----+
| PATRICK GARDNER|326373.19|           7|   5|
|BRYAN RUBENSTEIN| 249293.3|           8|   5|
|RUDY CASTELLANOS|195290.76|           9|   5|
|JOSHUA BAMBERGER|183754.57|          10|   5|
+----------------+---------+------------+----+

